In [1]:
import numpy as np
import logging
import synapses
syn_dir = 'components/synaptic_models'
synapses.load()
syn = synapses.syn_params_dicts(syn_dir=syn_dir)

from bmtk.builder.networks import NetworkBuilder
from bmtk.builder.auxi.node_params import positions_cuboid

n_E = 200 # number of RTM excitatory cells
n_I = 50  # number WB inhibitory cells
    

net = NetworkBuilder('network')

net.add_nodes(N=n_I, pop_name='PV',     # N = number of inhibitory cells
        model_type='biophysical',
        model_template='hoc:WBInhCell',  # WBInhCell hoc definition
        morphology='blank.swc',
        positions = positions_cuboid(N=n_I,center=[0.,0.,0.],xside_length=10.,yside_length=10.,height=10.,min_dist=2))

net.add_nodes(N=n_I, pop_name='SOM',     # N = number of inhibitory cells
        model_type='biophysical',
        model_template='hoc:WBInhCell',  # WBInhCell hoc definition
        morphology='blank.swc',
        positions = positions_cuboid(N=n_I,center=[0.,0.,0.],xside_length=10.,yside_length=10.,height=10.,min_dist=2))

net.add_nodes(N=n_E, pop_name='Exc',     # N = number of excitatory cells
        model_type='biophysical',
        model_template='hoc:RTMExcCell', # RTMExcCell hoc definition
        morphology='blank.swc',
        positions = positions_cuboid(N=n_E,center=[0.,0.,0.],xside_length=10.,yside_length=10.,height=10.,min_dist=2))


--No graphics will be displayed.


In [2]:
import random

def syn_connector(source,target,p):
    return 1 if random.random() < p else 0

In [3]:
p_EE = 0.0 # Probability of Excitatory RTM Cells Connecting to Excitatory RTM Cells
p_EI = 1.0 #0.5 # Probability of Excitatory RTM Cells Connecting to Inhibitory WB Cells
p_IE = 1.0 #0.5 # Probability of Inhibitory WB Cells Cells Connecting to Excitatory RTM Cells
p_II = 1.0 #0.5 # Probability of Inhibitory WB Cells Cells Connecting to Inhibitory WB Cells

g_hat_EE = 0.0
g_hat_EI = 1.25 #0.25
g_hat_IE = 1.25 #0.25
g_hat_II = 0.4  #0.25

conn = net.add_edges(source={'pop_name':'Exc'}, target={'pop_name':'Exc'},
            connection_rule=syn_connector,
            connection_params={'p':p_EI},
            syn_weight=g_hat_EE/n_E, # Synaptic weight is g^ over the total number of cell in the population
            dynamics_params='Exc2Exc.json',
            model_template='exp2syn',
            distance_range=[0.0,999.9],
            delay=0.5,
            target_sections=['soma'])

conn = net.add_edges(source={'pop_name':'Exc'}, target={'pop_name':'PV'},
            connection_rule=syn_connector,
            connection_params={'p':p_EI},
            syn_weight=(g_hat_EI/n_E)/3, # Synaptic weight is g^ over the total number of cell in the population
            dynamics_params='Exc2PV.json',
            model_template='exp2syn',
            distance_range=[0.0,999.9],
            delay=0.5,
            target_sections=['soma'])

conn = net.add_edges(source={'pop_name':'Exc'}, target={'pop_name':'SOM'},
            connection_rule=syn_connector,
            connection_params={'p':p_EI},
            syn_weight=g_hat_EI/n_E, # Synaptic weight is g^ over the total number of cell in the population
            dynamics_params='Exc2SOM.json',
            model_template='Exp2Syn_STP',
            distance_range=[0.0,999.9],
            delay=0.5,
            target_sections=['soma'])


In [4]:

conn = net.add_edges(source={'pop_name':'PV'}, target={'pop_name':'Exc'},
            connection_rule=syn_connector,
            connection_params={'p':p_IE},
            syn_weight=g_hat_IE/n_I, # Synaptic weight is g^ over the total number of cell in the population
            dynamics_params='PV2Exc.json',
            model_template='Exp2Syn_STP',
            distance_range=[0.0,999.9],
            delay=0.5,
            target_sections=['soma'])

conn = net.add_edges(source={'pop_name':'SOM'}, target={'pop_name':'Exc'},
            connection_rule=syn_connector,
            connection_params={'p':p_IE},
            syn_weight=g_hat_IE/n_I, # Synaptic weight is g^ over the total number of cell in the population
            dynamics_params='SOM2Exc.json',
            model_template='Exp2Syn_STP',
            distance_range=[0.0,999.9],
            delay=0.5,
            target_sections=['soma'])


conn = net.add_edges(source={'pop_name':'PV'}, target={'pop_name':'PV'},
            connection_rule=syn_connector,
            connection_params={'p':p_II},
            syn_weight=g_hat_II/n_I, # Synaptic weight is g^ over the total number of cell in the population
            dynamics_params='PV2PV.json',
            model_template='exp2syn',
            distance_range=[0.0,999.9],
            delay=0.5,
            target_sections=['soma'])

conn = net.add_edges(source={'pop_name':'SOM'}, target={'pop_name':'PV'},
            connection_rule=syn_connector,
            connection_params={'p':p_II},
            syn_weight=20, # Synaptic weight is g^ over the total number of cell in the population
            dynamics_params='SOM2PV.json',
            model_template='exp2syn',
            distance_range=[0.0,999.9],
            delay=0.5,
            target_sections=['soma'])



In [5]:
net.build()
net.save_nodes(output_dir='network')
net.save_edges(output_dir='network')

In [6]:
from bmtk.builder.networks import NetworkBuilder

extnet = NetworkBuilder('extnet')

extnet.add_nodes(N=n_E, model_type='virtual', pop_name='bgnoisevirtExt', pop_group='bgnoisevirtExt')


In [7]:
def target_ind_equals_source_ind(source, targets, offset=0, min_syn=1,max_syn=1):
    # Creates a 1 to 1 mapping between source and destination nodes
    total_targets = len(targets)
    syns = np.zeros(total_targets)
    target_index = source['node_id']
    syns[target_index-offset] = 1
    return syns
        

g_bar_stoch = 0.03
    
conn = extnet.add_edges(target=net.nodes(pop_name='Exc'),
            source={'pop_name':'bgnoisevirtExt'},
            iterator='one_to_all',
            connection_rule=target_ind_equals_source_ind,
            connection_params={'offset':n_I},
            dynamics_params='background_syn.json',
            model_template='exp2syn',
            distance_range=[0.0,999.9],
            target_sections=['soma'],
            delay=0.5,
            syn_weight=g_bar_stoch
            )

extnet.build()
extnet.save_nodes(output_dir='network')
extnet.save_edges(output_dir='network')

In [9]:
def generate_ext_poisson_input(n_E,freq=60,offset=0,population='extnet',times=(0.0, 5.0),filename='input/ext_inp_poisson.h5'):
    '''
    Generate poisson input for ext cells
    '''
    import os
    from bmtk.utils.reports.spike_trains import PoissonSpikeGenerator
    
    try:
        os.remove(filename)
    except OSError:
        pass
    
    psg = PoissonSpikeGenerator(population=population)
    psg.add(node_ids=range(offset,offset+n_E),  # Select nodes
            firing_rate=freq,     # 'freq' Hz, we can also pass in a nonhomoegenous function/array
            times=times)    # Firing starts at 0 s up to 1 s by default
    psg.to_sonata(filename)
    logging.info(filename + " written")


#n_E defined above
stoch_freq = 60

generate_ext_poisson_input(n_E,freq=stoch_freq,filename='input/ext_inp_poisson.h5')